In [1]:
import pandas as pd
import numpy as np

df16 = pd.read_csv('rawdata2016.csv')
df17 = pd.read_csv('rawdata2017.csv')
df18 = pd.read_csv('rawdata2018.csv')

In [4]:
# Combine data for each year
df = pd.concat([df16, df17, df18])
# What is "Unnamed: 0"?
df = df.drop(columns=['Unnamed: 0'])
df.head(15)

,CA,SCP,STATION,LINENAME,DATE,TIME,ENTRIES,EXITS
0,A002,02-00-00,59 ST,NQR456,02/27/2016,03:00:00,5562172,1877574
1,A002,02-00-00,59 ST,NQR456,02/27/2016,07:00:00,5562184,1877586
2,A002,02-00-00,59 ST,NQR456,02/27/2016,11:00:00,5562267,1877691
3,A002,02-00-00,59 ST,NQR456,02/27/2016,15:00:00,5562516,1877745
4,A002,02-00-00,59 ST,NQR456,02/27/2016,19:00:00,5562920,1877821
5,A002,02-00-00,59 ST,NQR456,02/27/2016,23:00:00,5563175,1877862
6,A002,02-00-00,59 ST,NQR456,02/28/2016,03:00:00,5563262,1877882
7,A002,02-00-00,59 ST,NQR456,02/28/2016,07:00:00,5563273,1877892
8,A002,02-00-00,59 ST,NQR456,02/28/2016,11:00:00,5563346,1877954
9,A002,02-00-00,59 ST,NQR456,02/28/2016,15:00:00,5563545,1878026


In [45]:
#stations = ['34 ST-PENN STA']
stations = ['CANAL ST', '33 ST', '23 ST', '14 ST', '28 ST', '34 ST-PENN STA']

def calc_delta(d):
    d['NEXT LINE TURNSTILE'] = d['SCP'].shift(-1)
    d['DELTA_ENTRY'] = np.where(d['NEXT LINE TURNSTILE'] == d['SCP'], d['ENTRIES'].shift(-1) - d['ENTRIES'], 0)
    d['DELTA_EXIT'] = np.where(d['NEXT LINE TURNSTILE'] == d['SCP'], d['EXITS'].shift(-1) - d['EXITS'], 0)
    d['TOTAL_DELTA'] = d['DELTA_ENTRY'] + d['DELTA_EXIT']
    len_before = len(d)
    d = d[(d['TOTAL_DELTA'] < 10000) &(d['TOTAL_DELTA']> 0)]
    len_after = len(d)
    len_lost = len_before - len_after
    print("{0:.0%}".format(len_lost/len_before), "rows unusable")
    return d

def calc_datetime(d):
    d["datetime_str"] = d.DATE + " " + d.TIME
    d["datetime"] = pd.to_datetime(d["datetime_str"], infer_datetime_format=True)
    d.drop(columns=["datetime_str"], inplace=True)
    d['day_of_week'] = d['datetime'].dt.weekday_name
    return d

In [46]:
# Make dict of dataframes for each station
frames = {}
for station in stations:
    print()
    print("Processing", station)
    d = df[df['STATION'] == station]
    d = calc_delta(d)  # Add DELTA columns for entry and exit
    d = calc_datetime(d)  # Combine date and time as datetime object
    d = d.dropna()
    # Separate weekend and weekday data
    d_wknd = d[(d['day_of_week'] == 'Saturday') | (d['day_of_week'] == 'Sunday')]
    frames[station+" wknd"] = d_wknd
    d_wkdy = d[(d['day_of_week'] != 'Saturday') & (d['day_of_week'] != 'Sunday')]
    frames[station+" wkdy"] = d_wkdy
    # Show ratio of weekday to weekend data
    print(len(d_wkdy)/len(d), "wkdy vs.", len(d_wknd)/len(d), "wknd")


Processing CANAL ST
11% rows unusable


/home/jessica/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
/home/jessica/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/jessica/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas

0.7110511102646261 wkdy vs. 0.28894888973537386 wknd

Processing 33 ST
12% rows unusable
0.7135527618590417 wkdy vs. 0.28644723814095835 wknd

Processing 23 ST
17% rows unusable
0.7216614296936371 wkdy vs. 0.27833857030636294 wknd

Processing 14 ST
7% rows unusable
0.7108544428402678 wkdy vs. 0.2891455571597322 wknd

Processing 28 ST
13% rows unusable
0.7259851000807827 wkdy vs. 0.2740148999192173 wknd

Processing 34 ST-PENN STA
10% rows unusable
0.7188272377687474 wkdy vs. 0.2811727622312527 wknd


    CANAL ST 0.7167051960035321 wkdy vs. 0.2832948039964679 wknd
    33 ST 0.7158699708301579 wkdy vs. 0.28413002916984204 wknd
    23 ST 0.7165942189150029 wkdy vs. 0.2834057810849972 wknd
    14 ST 0.715475514453368 wkdy vs. 0.28452448554663207 wknd
    28 ST 0.7164351671015421 wkdy vs. 0.2835648328984579 wknd

For each station, about 72% of the data was collected on a weekday and 28% was collected on weekends. We're graphing them separately because traffic might be drastically different due to the typical work/school schedule.

In [47]:
# Find total traffic for each entrance of station (sum of SCP).
def groupby_sum(station):
    return station.groupby(by=['CA', 'STATION', 'DATE', 'TIME']).sum()

# Average hourly traffic for each entrance of station.
def groupby_mean(station):
    return station.groupby(by=['CA', 'STATION', 'TIME']).mean()

In [48]:
for key, station in frames.items():
    station = groupby_sum(station)
    station = groupby_mean(station)
    station.sort_values(by=['TIME'], inplace=True)
    fn = key.replace(" ", "_") + ".csv"
    station.to_csv(fn)
    print("Completed", fn)

Completed CANAL_ST_wknd.csv
Completed CANAL_ST_wkdy.csv
Completed 33_ST_wknd.csv
Completed 33_ST_wkdy.csv
Completed 23_ST_wknd.csv
Completed 23_ST_wkdy.csv
Completed 14_ST_wknd.csv
Completed 14_ST_wkdy.csv
Completed 28_ST_wknd.csv
Completed 28_ST_wkdy.csv
Completed 34_ST-PENN_STA_wknd.csv
Completed 34_ST-PENN_STA_wkdy.csv
